# Profile likelihood ratio calculator

Copyright (c) 2020, Pietro Vischia pietro.vischia@cern.ch


### Implementation of a class for profile likelihood calculations, with and without systematic uncertainties
Based on MINOS via iMinuit (Fred James' still rules)

Maybe in the future port to tensorflow?

##### Features
- [x] Profiling
- [x] Constraints on nuisance parameters
- [ ] Pulls and impacts
- [ ] Breakdown of systematic uncertainties


In [1]:
# Check profile likelihood
import numpy as np
import scipy as sp
import scipy.stats as st
from scipy import optimize as op
import matplotlib
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math
import sys

import random
import inspect
from iminuit import Minuit
import iminuit
print(iminuit.__version__)

1.5.4


Let's see a simple class that calculates a profile likelihood ratio given a minus-log-likelidood.

It supports freezing parameters to prefit values, and setting up Gaussian constraints for nuisance parameters

In [47]:
class profile_likelihood_ratio():
    def __init__(self, model, data, ts, minuslogmodel=None, freezelist=None,nsteps=100, migradprecision=None):
        self.model = model
        self.minuslogmodel = minuslogmodel
        self.n_params=len(self.model.__code__.co_varnames)-2
        self.data = data
        self.ts=ts
        self.migradprecision= migradprecision
        self.constraints = {}
        self.initial_values=[1. for i in range(self.n_params)]
        #self.likelihood_realization=lambda params : -2*(self.model(self.ts(self.data), params))
        self.likelihood_realization=lambda params : (self.model(self.data, params)) # I save it but I DO NOT USE IT
        self.minusloglikelihood_realization= lambda params: 2*self.minuslogmodel(self.data, params) #if self.minuslogmodel else -2*np.log(self.model(self.data, params))
        # Work with variable names, to exploit <function>.__code__.co_varnames to have arbitrary number of variables
        self.pardict = { parname : 1 for parname in self.minuslogmodel.__code__.co_varnames if parname!='data' and parname!='params'} if self.minuslogmodel else { parname : 1 for parname in self.model.__code__.co_varnames if parname!='data' and parname!='params'}
        self.freezelist= freezelist if freezelist else { parname : [False, None, False] for parname in self.pardict.keys()}
        self.initial_values = [ 1. for i in range(self.n_params)]
        self.mles = None
        self.mleuncs = None
        self.nsteps = nsteps # For plotting
    def __call__(self, params):
        # I need the class to be callable, in order to call it. Sweet.
        return self.minusloglikelihood_realization(params)
        
    def minimize(self):
        # scipy.optimize.minimize is so crappy that I switched to iMinuit, and am happy
        #ret = op.minimize(self, self.initial_values, constraints=self.constraints )

        # Errordef should be 0.5 for -lnL, and 1 for least_squares, according to iminuit tutorial
        # However, I am already multiplying -lnL by 2, so the correct errordef value is 1.

        # Instance the minimizer
        m = Minuit.from_array_func(plr, self.initial_values, name=self.pardict.keys(), errordef=1) 
        m.print_level=3 # 0: silent. 3: even low-level Minuit messages
        
        # Take care of frozen parameters and constrain nuisance parameters
        for par, freezestatus in self.freezelist.items():
            print('par %s has freezestatus %s'%(par, freezestatus))
          
            if freezestatus[1] is not None: # Set the input value of parameters that need to be frozen to their input value
                m.fitarg[par] = freezestatus[1]
                m.values[par] = freezestatus[1]
                m.fitarg['error_%s'%par] = 0.
                print('I am freezing (conditioning on) parameter %s to %s'%(par, freezestatus[1]))
            if freezestatus[2]: # Nuisance parameters that need to be constrained
                m.fitarg[par] = ...### FILL THIS
                m.values[par] = ....### FILL THIS
                m.fitarg['error_%s'%par] = ...### FILL THIS
                print('I am settting the initial value of... ')
            
            # Freeze all parameters that need to be frozen
            m.fitarg['fix_%s'%par] = freezestatus[0] # Freeze it
            m.fixed[par] = freezestatus[0]
            print('Minuit model settings:', m.fitarg)
            
        print('Minimizer will run with the following model settings: ', m.fitarg)
        
        # Minimize
        if self.migradprecision:
            m.migrad(precision=self.migradprecision)
        else:
            m.migrad()
        
        m.hesse()
        
        # Covariance matrix
        print(m.covariance)
        # Correlation matrix
        # m.covariance.correlation()  # Minuit has changed interface inbetween versions?
        
        m.minos(sigma=1) # Compute asymmetric uncertainties (better than m.hesse() ). Fred James rulezzz
        print('I have run MINOS')
        print(m.params)
        
        # Minos profile
        m.draw_mnprofile(list(self.pardict.keys())[0],subtract_min=True) # Draw MINOS intervals                                                                                                                                                                                                                 
        #m.draw_profile(list(self.pardict.keys())[0],subtract_min=True); # Draws Hesse intervals
                                 
        
        return [m.values, m.merrors] # when calling m.minos()
        #return [m.values, m.errors] # when calling only m.hesse()
        
    def do_global_fit(self):
        
        # Main fit
        results=self.minimize()
        
        # Look at results
        print('Main fit Results', results)
        print('Main fit Best estimates', results[0])
        print('Main fit Uncertainties', results[1])
        self.mles = { par : results[0][par] for par in self.pardict.keys() }
        self.mleuncs = {}
        for par in self.pardict.keys():
            if self.freezelist[par][0] == False:
                self.mleuncs[par, +1] = results[1][par, +1.0]
                self.mleuncs[par, -1] = results[1][par, -1.0]

        for par, val in self.mles.items():
            if self.freezelist[par][0] == False:
                print('Parameter %s = %s (+%s/-%s) [fitted]'%(par,val, self.mleuncs[par, +1],self.mleuncs[par, -1]))
            else:
                print('Parameter %s = %s [frozen]'%(par,val))
    
    def compute_model_at(self, datapoint, pars):
        print(self.model(datapoint, pars))
        
    def plot_results(self, scan):
        pass
        
    def find_range_around_mle(self, par):
        mle = self.mles[par]
        # Wild guess, must find a more meaningful way (maybe pre-finding the range in which it's nonzero)
        return [mle+5*self.mleuncs[par, -1], mle+5*self.mleuncs[par, +1]] if self.freezelist[par][0] == False else [mle-5*abs(mle), mle+5*abs(mle)]
    
    def draw_data(self):
        fig = plt.figure(figsize=(10,10))
        plt.hist(self.data, bins=100)
        thepar=list(self.pardict.keys())[0] # Assume the location parameter is kind of the first parameter
        plt.axvline(x=self.ts(self.data), ls='--', c='red', label='Test statistic value')
        plt.axvline(x=self.mles[thepar], ls='--', c='black', label='Maximum likelihood estimate for location parameter theta')
        plt.axvspan(self.mles[thepar]+self.mleuncs[thepar, -1],self.mles[thepar]+self.mleuncs[thepar,+1], alpha=0.5, color='grey', label='68.3% CL interval')

        plt.legend(loc='best')
        
    def draw_likelihood(self, override_range=None):
        ret=[]
        # Draw the 1D likelihoods
        # This doesn't work properly yet: it will be the nucleus of the systematic breakdown by freezing stuff
        # but it still has undebugged issues
        for par in self.pardict.keys():
            print('NOW PLOTTING %s'%par)
            par_idx = list(self.pardict.keys()).index(par)
            fig = plt.figure(figsize=(10,10))
            scanrange = self.find_range_around_mle(par)
            if override_range: # I can't remember what is this
                if override_range[par]:
                    scanrange=override_range[par]

            # Pick the MLES for floating parameters, and the input values for the frozen ones
            theparams=[ [ j for j in self.initial_values] for i in range(self.nsteps) ]
            conditionlist = { conditionedpar : None for conditionedpar in self.pardict.keys() if conditionedpar!= par  }
            
            for i_params in range(self.nsteps):
                for mledpar in self.pardict.keys():
                    if mledpar != par:
                        mledpar_idx=list(self.pardict.keys()).index(mledpar)
                        theparams[i_params][mledpar_idx] = self.mles[mledpar] 
                        if self.freezelist[mledpar][0]:
                            theparams[i_params][mledpar_idx] = self.freezelist[mledpar][1]
                        conditionlist[mledpar] = theparams[i_params][mledpar_idx]
     
            print('Plotting %s in range %s in %s steps'%(par, scanrange, self.nsteps))            
            x=np.linspace(scanrange[0], scanrange[1], num=self.nsteps)
            
            for idx in range(self.nsteps):
                #print(idx, x[idx], theparams[idx][par_idx])
                theparams[idx][par_idx] = x[idx]
                #print('\t', idx, x[idx], theparams[idx][par_idx])


            y= [self(params) for params in theparams ]
            #print('Theparams', theparams)
            #for my_idx in range(self.nsteps):
            #    print(my_idx, theparams[my_idx], y[my_idx])
            #print('linspace x', x)
            plt.plot(x,y-np.nanmin(y), label='Delta ln(L) scan')
            plt.axvline(x=self.mles[par], ls='--', c='red', label='Maximum likelihood estimate')
            plt.axhline(y=1, c='blue', ls='--', alpha=0.5, label='68.3% CL')
            plt.axhline(y=4, c='blue', ls='--', alpha=0.5, label='95% CL')
            #1D: 1, 4; 2D: 2.29, 5.99)
            if self.freezelist[par][0] == False:
                plt.axvspan(self.mles[par]+self.mleuncs[par, -1],self.mles[par]+self.mleuncs[par,+1], alpha=0.5, color='grey', label='68.3% CL interval')
            plt.xlabel('Scan of %s while fixing %s'%(par, [[key, value] for key, value in conditionlist.items()]))
            plt.ylabel('Delta ln(L)')
            plt.ylim(-1,10)
            plt.legend(loc='best')
            ret.append(fig)
        return ret
        
    def print_inputs(self, alsoData=False):
        if(alsoData):
            print('Data:')
            print(data)
        print('Test statistic: ')
        print(self.ts)
        print('Test statistic value:')
        print(self.ts(self.data))
        print('Parameters:')
        print(self.model.__code__.co_varnames)
        print('Parameters dictionary:')
        print(self.pardict)
        print('Initial values:')
        print(self.initial_values)
        print('Model:')
        print(inspect.getsource(self.model))
        print('[UNUSED] Likelihood model (fixed data):')
        print(inspect.getsource(self.likelihood_realization))
        print('[UNUSED] Likelihood model parameters:')
        print(self.likelihood_realization.__code__.co_varnames)
        print('minus-log-Likelihood model (fixed data):')
        print(inspect.getsource(self.minusloglikelihood_realization))
        print('minus-log-Likelihood model parameters:')
        print(self.minusloglikelihood_realization.__code__.co_varnames)
        #print(self.model)


Let's start with a Gaussian likelihood, of which we attempt to estimate both the mean and the variance.

The datasets is sampled from a Gaussian with $\theta_{true}=5$ and $\sigma_{\true}=2$.

You need to write down the minus-log-likelihood function. Look at the profiled regions.

You can play with the number of data and with the true values used to generate the data.

In [ ]:
# First let's start with a gaussian likelihood
def my_gaussian_likelihood(data, params):
    pass

def my_gaussian_minusloglikelihood(data, params):
    theta=params[0]
    sigma=params[1]
    return # THE VALUE OF THE MINUS LOG LIKELIHOOD


Ntot=10000
data = np.array([ random.gauss(5,2) for i in range(Ntot)])
debug=True

plr = profile_likelihood_ratio(my_gaussian_likelihood, data, np.mean, my_gaussian_minusloglikelihood)

if debug:
    plr.print_inputs()
plr.do_global_fit()

Visualize the data, the true value, and the sampling mean

In [ ]:
figs = plr.draw_data()

plt.show()


# figs = plr.draw_likelihood({'theta' : None, 'sigma' : None}) # this doesn't work properly yet
#for fig in figs:
#    plt.show()

#x = np.linspace(0,10)
#y = [ my_gaussian_likelihood(data[0], [x_i, 1]) for x_i in x]
#plt.plot(x, y)
#plt.show()

Now you have to write a Poisson model for the case of a signal and one background.

I remind you that you need to parameterize the signal yield in terms of signal strength $\mu = \frac{\sigma}{\sigma_{expected}}$

The model is:
    
$$
p(X|\mu,s,b)= Pois(X | \mu s + b)
$$

And that to write down the likelihood you need to condition on the observed data.

In [ ]:
def my_poisson_count_model_1bin_nonuisance(data, params):
    pass

def my_poisson_count_model_1bin_nonuisance_minusloglikelihood(data, params): # analytical loglikelihood better computationally (small values)
    mu=params[0] # POI
    s=params[1] 
    b=params[2]
    
    return ### FILL HERE THE VALUE OF THE MINUS LOG LIKELIHOOD



Nobs=20
sexp=5
bexp=15

#print(inspect.getsource(my_marked_poisson_model_1bin_nonuisance))

x=np.linspace(0,4, num=200) # scan mu
y = [ my_poisson_count_model_1bin_nonuisance( Nobs, [x_i, sexp, bexp] ) for x_i in x  ]

plt.plot(x,y, label='L( %s | mu*%s +%s)' %(Nobs, sexp, bexp))
plt.xlabel('mu = s/s_exp')
plt.ylabel('L(%s | mu*%s + %s)'%(Nobs, sexp, bexp))
plt.legend(loc='best')
plt.show()



Let's look at the likelihood for different values of the expected signal $s$ and background  $b$


In [ ]:

fig = plt.figure()
ax = fig.gca(projection='3d')

yy = np.linspace(1,10, num=10) # scan sexp
xx = np.linspace(0,4, num=10) # scan mu
zz = np.zeros( (xx.size, yy.size) )
counter_y = 0
for X in xx:
    counter_x = 0
    for Y in xx:
        zz[counter_x,counter_y] = my_poisson_count_model_1bin_nonuisance(Nobs, [X, Y, bexp])
        counter_x += 1
    counter_y += 1
#zz=[ [ my_marked_poisson_model_1bin_nonuisance(Nobs, [X, Y, bexp]) for Y in yy ] for X in xx]
print(len(xx), len(yy), len(zz))
X, Y = np.meshgrid(xx,yy)
#zz = my_marked_poisson_model_1bin_nonuisance( Nobs, [X, Y, bexp] ) 
#mmpm1n = np.vectorize(my_marked_poisson_model_1bin_nonuisance)
#zz= mmpm1n( Nobs, [X, Y, bexp])
#print(my_marked_poisson_model_1bin_nonuisance( Nobs, [0.88888889, 3, bexp] ) )
#zz = [ my_marked_poisson_model_1bin_nonuisance( Nobs, [xx_i, yy_i, bexp] ) for xx_i, yy_i in zip(X, Y) ]
ax.plot_wireframe(X, Y, zz )
ax.set_xlabel('mu = s/s_exp')
ax.set_ylabel('Expected signal sexp')
#ax.set_zlim(0, 0.0000001)
#Nobs=45
#debug=True

#plr = profile_likelihood_ratio(my_gaussian_likelihood, data, np.mean, my_gaussian_minusloglikelihood)

#if debug:
#    plr.print_inputs()
#plr.do_global_fit()

Run the likelihood fit

In [ ]:
# Now try   profile likelihood ratio routine with this simple model
Ntot=1000
data = st.poisson.rvs(20, size=Ntot)
data = 20
debug=True


# 'mu': [ mustBeFrozen, value to freeze to, isNuisanceParameter],
freezelist = {'mu': [ FILL HERE],
              's': [ FILL HERE ],
              'b': [FILL  HERE ]}

plr = profile_likelihood_ratio(my_poisson_count_model_1bin_nonuisance, data, np.mean, my_poisson_count_model_1bin_nonuisance_minusloglikelihood, freezelist)

if debug:
    plr.print_inputs()
plr.do_global_fit()

Now we want to add a nuisance parameter and model it as a Gaussian distribution.

We encode a nuisance parameter $\alpha$ acting on the background yield (uncertainty in the background normalization) as 

$$
\mathcal{L}(\mathbf{n}, \mathbf{\alpha^0} | \mu, \mathbf{\alpha}) = \prod_{i\in bins} \mathcal{P}(n_i | \mu S_i(\mathbf{\alpha}) \
+ B_i(\mathbf{\alpha}))\times \prod_{j\in syst} \mathcal{G}(\alpha_j^0 | \alpha_j, \delta\alpha_j)$\\
$$

And end up rescaling it such that the Gaussian is centered at 0 and with unit variance

$$
\mathcal{L}(\mathbf{n}, 0 | \mu, \mathbf{\alpha}) = \prod_{i\in bins} \mathcal{P}(n_i | \mu S_i(\mathbf{\alpha}) + B_i(\mathbf{\alpha}))\times \prod_{j\in syst} \mathcal{G}(0 | \alpha_j, 1)
$$


In [ ]:
# Now let's try to add a nuisance parameter.

# First let's have one bin (channel) and check it works
def my_poisson_count_model_1bin(data, params):
    pass


def my_poisson_count_model_1bin_minusloglikelihood(data, params): # analytical loglikelihood better computationally (small values)
    mu=params[0] # POI
    s=params[1] # We will condition on this
    b=params[2] # We will condition on this
    bunc=params[3] # We will profile this

    return ### FILL HERE THE VALUE OF THE MINUS LOG LIKELIHOOD


Nobs=20
sexp=5
bexp=15
bunc=0.2
x=np.linspace(0,4, num=200) # scan mu


y = [ my_poisson_count_model_1bin_minusloglikelihood( Nobs, [x_i, sexp, bexp, bunc] ) for x_i in x  ]
#y = my_poisson_count_model_1bin(Nobs, [x, sexp, bexp, bunc])
plt.plot(x,y, label='Pois( %s | mu*%s + %s(1+%s*%s))*Gaus(0|%s,1)' %(Nobs, sexp, bexp, bunc, bexp, bunc))
plt.xlabel('mu = s/s_exp')
plt.ylabel('L(%s | mu*%s + %s(1+%s*%s))*Gaus(0|%s,1)'%(Nobs, sexp, bexp, bunc, bexp,bunc))
#plt.ylim(0,0.12)
plt.legend(loc='best')
plt.show()



Now profile it!

In [ ]:
Ntot=1000
#data = st.poisson.rvs(20, size=Ntot)

debug=True

data = 50
sexp=5
bexp=15
bunc=0.0002

freezelist = {'mu': [ FILL HERE ],
              's': [ FILL HERE],
              'b': [FILL HERE],
              'bunc': [FILL HERE]}

plr = profile_likelihood_ratio(my_poisson_count_model_1bin, data, np.mean, minuslogmodel=my_poisson_count_model_1bin_minusloglikelihood, freezelist=freezelist,migradprecision=0.0001)
#migradprecision=0.1
if debug:
    plr.print_inputs()
plr.do_global_fit()
plr.draw_likelihood()
plr.draw_data()